# ISS_preprocessing of Leica-exported tiffs

This notebook guides you through the preprocessing of tiff files with associated Metadata as exported by Leica microscopes. **The sofware is not able and it is not designed to process Leica proprietary formats such as .lif .lof and the likes.**  

To use this notebook the you must have exported, directly from the Leica software, the files as individual TIFFs. Each tiff file will represent a single plane of a single tile of a single channel, so thousands of individual TIFFs will be created in a typical experiment. 

For our script to work it is important that the files are rigorously and consistently indexed, and this guaranteed by the following example naming structure:

`TileScan 1--Stage99--Z42--C03.tif`, where 

- `TileScan` represents the ROI, 

- `Stage` represents the tile, 

- `Z` represent Z-plane,

- `C` represents the channel, and  

- the `--`  sign acts as a separator (default on our Leica microscopes).

We begin importing the necessary libraries and tools

In [ ]:
import ISS_processing.preprocessing as pp
import os
import pandas as pd

## Main function for Leica processing.
This function is a sort of main wrapper to process Leica-exported tiffs, and under the hood runs many functions that are made invisible for convenience. Once if you figured out the right parameters for your specific microscope, this will be the smoother way of running preprocessing.

In the function you will have to define the following variables:


`input_dirs`: This will be a list of the complete paths to the folders containing your imaging cycles. The folders need to be specified in the right order (ie. the first element of the list will be the folder where the first cycle of imaging is saved, and so on) The elements need to be separated by commas. The format for this variable is a `lst` of `str`.

`output_location` = This will be the path where you want the preprocessing output to be saved. Ideally, this should be associated with some type of unique project identifier. The format of this variable is `str`. In case multiple regions are being processed, the funcion will behave differently depending whether a trailing slash `/` is included or not in `output_location`: 
- if a trailing slash is added, then subfolders for each one of the scanned regions will be created as `_R1`, `_R2`, etc...
- if a trailing slash is omitted, then `output_location` will be updated at each region iteration, and each one of the scanned regions will end up in a different `output_location` named as `output_location_R1`,`output_location_R2`, etc...

`regions_to_process`: This is an `int` refering to the number of regions of interest (ROI) that you have in each folder. 

`align_channel`: This variable sets on which channel the alignment across cycles is performed, and typically points to the DAPI channel. This is an `int` number, which referes to the channel number containing the DAPI images. The number is relative to the order of channel acquisition on your  microscope. In our Leica set up, this is the fifth channel, which in python would mean that we put 4 (since python is zero indexed).  

`tile_dimensions` =  This `int` refers to the number of pixels that you want to tile your images into during the reslicing process. I.e. if you specify 4000, your resliced images will be of the shape 4000x4000 pixels. 

`mip`:  This is a `boolean` that specifies whether or not you want to run the Maximum-projection step. From some microscope you might prefer to export already projected images. Or maybe you decided to perform a preliminary image deconvolution and save the deconvolved images as maximum projection. In this case set `mip='False'`, otherwise leave it to 'True'. 

If you specify `mip='False'`, one important thing has to be noted: in the main function `input_dirs` are only used to extract the raw stacks and maximum project them. If you choose to start from projected images, the next steps use as input the `output_location` folder and its subfolders. So in case `mip='False'`, your files need to be stored and appropriately named in the `output_location` folder, more specifically under the `/preprocessing/mipped/` subfolder to be found by the function. Please refer to the manual if this is not clear.


In [ ]:
pp.preprocessing_main_leica(input_dirs = ['/path/to/cycle1/',
                                       '/path/to/cycle2/',
                                       '/path/to/cycle3/'], 
                output_location = '/path/to/output_folder/',
                regions_to_process = 1, 
                align_channel = 5, 
                tile_dimension = 6000, 
                mip = True)

## Access to individual functions for Leica processing.

Instead of running the main function as outlined above, we can also choose to run the step by step subfunctions one at a time. Let's have a quick look at what each function does.


### `leica_mipping`

`leica_mipping` is the function to **organise and maximum-project the images from the input folders**. It takes the following arguments, which mirror the same arguments of the main function:

`input_dirs`: This will be a list of the complete paths to the folders containing your imaging cycles. The folders need to be specified in the right order (ie. the first element of the list will be the folder where the first cycle of imaging is saved, and so on) The elements need to be separated by commas. The format for this variable is a `lst` of `str`.

`output_location` = This will be the path where you want the preprocessing output to be saved. Ideally, this should be associated with some type of unique project identifier. The format of this variable is `str`. In case multiple regions are being processed, the funcion will behave differently depending whether a trailing slash `/` is included or not in `output_location`: 
- if a trailing slash is added, then subfolders for each one of the scanned regions will be created as `_R1`, `_R2`, etc...
- if a trailing slash is omitted, then `output_location` will be updated at each region iteration, and each one of the scanned regions will end up in a different `output_location` named as `output_location_R1`,`output_location_R2`, etc...

`leica_mipping` is able to handle multiple regions in the input files, and project them accordingly.

In [ ]:
from ISS_processing import preprocessing 

pp.leica_mipping(input_dirs = ['/path/to/cycle1/',
                                       '/path/to/cycle2/',
                                       '/path/to/cycle3/'], 
                output_dir_prefix = '/path/to/output_folder/')

### `leica_OME_tiff`
`leica_OME_tiff` is the function that **takes the projected images across channels and wraps them into a single OMEtiff per imaging cycle**. This steps organises the files corresponding to each imaging cycles in a specific way within a single file and requires the parsing of a Metadata file to arrange correctly the images in xy space. As explained before, the input for this function is the specific sub-folder within `output_location` one wishes to process. **This function does not allow to process multiple regions in one go, and needs to be run on individual regions manually**.

The function outputs 1 OMEtiff file per cycle into the `/preprocessing/OME_tiffs/` subfolder.



In [ ]:
from ISS_processing import preprocessing 

path = '/path/to/output_folder/region/'
# create leica OME_tiffs
pp.leica_OME_tiff(
    directory_base = path+'/preprocessing/mipped/',
    output_directory = path+'/preprocessing/OME_tiffs/'
)

### `ashlar_wrapper`

This function runs `ashlar`, a package for image stitching and cycle alignment. The function uses the OME_tiffs files as an input, takes as input a channel number (normally the DAPI, see above) and on that channels performs all the alignment and stitching operations.

The function outputs 1 stitched file per cycle and channel into the `/preprocessing/stitched/` subfolder

In [ ]:
from ISS_processing import preprocessing 

# align and stitch images
path = '/path/to/output_folder/region/'
OME_tiffs = os.listdir(path+'/preprocessing/OME_tiffs/')
OME_tiffs = [path+'/preprocessing/OME_tiffs/' + sub for sub in OME_tiffs]

pp.ashlar_wrapper(
    files = OME_tiffs,
    output = path+'/preprocessing/stitched/',
    align_channel=4
)


### `tile_stitched_images`
In this function the stitched images are re-tiled according to a user-specified size.
The reason for this is that stitched images are too big to be decoded directly and we prefer to decode them in tiles. This has several advantages, most notably that the pipeline would work also on laptops or non-powerful computers. The idea tile size is 4000-6000, but larger or smaller are also fine depending on the computer.

In [ ]:
from ISS_processing import preprocessing 

# align and stitch images
path = '/path/to/output_folder/region/'

# retile stitched images
pp.tile_stitched_images(
    image_path = path+'/preprocessing/stitched/',
    outpath = path+'/preprocessing/ReslicedTiles/',
    tile_dim=2000
)

